# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Batch Inference</span>


## 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
feature_view = fs.get_feature_view(
    name='transactions_view_fraud_batch_fv',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🚀 Fetch and test the model</span>

Finally you can start making predictions with your model! Retrieve your model from Hopsworks model registry.

In [ ]:
retrieved_model = mr.get_model(
    name="xgboost_fraud_batch_model",
    version=1,
)
saved_model_dir = retrieved_model.download()

In [ ]:
retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_fraud_batch_model.pkl")
retrieved_xgboost_model

---
## <span style="color:#ff5f27;">🔮  Batch Prediction </span>


In [ ]:
feature_view.init_batch_scoring(1)

batch_data = feature_view.get_batch_data()

batch_data.drop(["datetime"], axis=1, inplace=True)

batch_data.head()

In [ ]:
predictions = retrieved_xgboost_model.predict(batch_data)

predictions[:5]

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Fraud Batch tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai